In [2]:
from keras.applications import VGG16

conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

Using TensorFlow backend.


In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [4]:
# A. 在数据集中运行conv_base卷据基，并将输出保存在硬盘，然后将输出作为输入用于新模型(不使用数据增强)

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/home/alex/下载/学习/data/dogs-vs-cats-small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [5]:
train_dir

'/home/alex/下载/学习/data/dogs-vs-cats-small/train'

In [6]:
datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 20

def extract_features(directory, sample_count):
    print('目录: %', directory)
    features = np.zeros(shape = (sample_count, 4, 4, 512))
    labels = np.zeros(shape = (sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary'
    )
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i % 100 == 0:
            print(i)
        if i * batch_size >= sample_count:
            print('break')
            break
    return features, labels


train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

目录: % /home/alex/下载/学习/data/dogs-vs-cats-small/train
Found 2000 images belonging to 2 classes.
100
break
目录: % /home/alex/下载/学习/data/dogs-vs-cats-small/validation
Found 1000 images belonging to 2 classes.
break
目录: % /home/alex/下载/学习/data/dogs-vs-cats-small/test
Found 1000 images belonging to 2 classes.
break


In [7]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [9]:
# 定义并训练密集连接分类器
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_features, train_labels, epochs=30, batch_size=20, validation_data=(validation_features,validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5896 - acc: 0.6805 - val_loss: 0.4294 - val_acc: 0.8290
Epoch 2/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4287 - acc: 0.8070 - val_loss: 0.3512 - val_acc: 0.8670
Epoch 3/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3545 - acc: 0.8515 - val_loss: 0.3181 - val_acc: 0.8810
Epoch 4/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3133 - acc: 0.8715 - val_loss: 0.3002 - val_acc: 0.8900
Epoch 5/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2854 - acc: 0.8900 - val_loss: 0.2867 - val_acc: 0.8840
Epoch 6/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2669 - acc: 0.8965 - val_loss: 0.2801 - val_acc: 0.8860
Epoch 7/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2500 - acc: 0.9075 - val_loss: 0.2687 - val_acc: 0.8940
Epoch 8/30
200

In [10]:
import matplotlib.pyplot as plt 

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Train Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Train & Validation accuracy')
plt.legend()

plt.savefig("dog_vs_cat_acc_VGG16_A.png")

plt.figure()

plt.plot(epochs, loss, 'bo', label='Train Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Train & Validation loss')
plt.legend()

plt.savefig("dog_vs_cat_loss_VGG16_A.png")
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [1]:
model.save('cats_and_dogs_small_VGG16_A.h5')

NameError: name 'model' is not defined

In [ ]:
# B 使用数据增强的特征提取 --》 扩展conv_base模型，在cpu上基本没戏
